# Лаба 2

In [2]:
import pandas as pd
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
test_path = r'/content/test.csv'
train_path = r'/content/train.csv'

test_df = pd.read_csv(test_path)
train_df = pd.read_csv(train_path)

In [4]:
train_df = train_df.head(40000)

In [5]:
train_df

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
39995,2,Schumi: #39;Not my weekend #39;,The seven-times Formula One champion spun out ...
39996,2,Australia leads 4-1 in Davis Cup relegation tie,Australia #39;s Todd Woodbridge #39;s straight...
39997,1,Newspaper tip-off leads to terror arrests,Police have been given more time to question f...
39998,1,Sagan dies at 69,"Francoise Sagan, the rebellious French writer ..."


In [6]:
test_df

,ID,Title,Description
0,0,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,1,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,2,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,3,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...
...,...,...,...
7595,7595,Around the world,Ukrainian presidential candidate Viktor Yushch...
7596,7596,Void is filled with Clement,With the supply of attractive pitching options...
7597,7597,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...
7598,7598,5 of arthritis patients in Singapore take Bext...,SINGAPORE : Doctors in the United States have ...


Признаки: Title и Description. Таргет: Class Index.

In [7]:
train_df['Class Index'].unique()

array([3, 4, 2, 1])

Задача классификации.          
Мир, спорт, бизнес и наука-технологии. (Всего 4)

Условия         
1) Попробуйте некоторые методы очистки текста из списка ниже (минимум 2). Улучшило ли что-то из этого качество вашей модели? Попробуйте предположить, почему.            
  /удаление стоп-слов;       
  /удаление знаков препинания;       
  /Корзина (удаление лишних пробелов/специальных символов (типа @#~<> и т.д.));                   
  /удаление цифр.             
2) Попробуйте применить стемминг и лемматизацию. Что лучше улучшит качество модели?           
3) Попробуйте разные типы векторизации и сравните модели, обученные с помощью разных типов векторизации:           
  /ТФ-ИДФ;          
  /Word2Vec или GloVe или FastText;       
  /Через RNN, Трансформер.            
4) Напишите свою собственную модель. Сравните качество MLP, RNN, CNN для указанной задачи. (минимум 2 из них)        
5) Проведите анализ ошибок вашей лучшей модели, выдвиньте гипотезу, как ее улучшить. Проведите эксперименты, чтобы проверить ее.       

## 1 задание. Очистка текста

1) Удаление знаков препинания

In [8]:
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)

# Функция для удаления знаков препинания
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

# Применение функции к столбцам Title и Description
train_df['Title'] = train_df['Title'].apply(remove_punctuation)
train_df['Description'] = train_df['Description'].apply(remove_punctuation)

test_df['Title'] = test_df['Title'].apply(remove_punctuation)
test_df['Description'] = test_df['Description'].apply(remove_punctuation)

In [9]:
train_df

,Class Index,Title,Description
0,3,Wall St Bears Claw Back Into the Black Reuters,Reuters Shortsellers Wall Streets dwindlingba...
1,3,Carlyle Looks Toward Commercial Aerospace Reuters,Reuters Private investment firm Carlyle Group...
2,3,Oil and Economy Cloud Stocks Outlook Reuters,Reuters Soaring crude prices plus worriesabou...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters Authorities have halted oil exportflo...
4,3,Oil prices soar to alltime record posing new m...,AFP Tearaway world oil prices toppling record...
...,...,...,...
39995,2,Schumi 39Not my weekend 39,The seventimes Formula One champion spun out i...
39996,2,Australia leads 41 in Davis Cup relegation tie,Australia 39s Todd Woodbridge 39s straight set...
39997,1,Newspaper tipoff leads to terror arrests,Police have been given more time to question f...
39998,1,Sagan dies at 69,Francoise Sagan the rebellious French writer w...


2) Удаление цифр

In [10]:
# Функция для удаления знаков препинания и чисел
def clean_text(text):
    # Удаляем знаки препинания и числа
    return re.sub(r'[0-9]', '', text)

# Применение функции к столбцам Title и Description
train_df['Title'] = train_df['Title'].apply(clean_text)
train_df['Description'] = train_df['Description'].apply(clean_text)

test_df['Title'] = test_df['Title'].apply(clean_text)
test_df['Description'] = test_df['Description'].apply(clean_text)

In [11]:
train_df

,Class Index,Title,Description
0,3,Wall St Bears Claw Back Into the Black Reuters,Reuters Shortsellers Wall Streets dwindlingba...
1,3,Carlyle Looks Toward Commercial Aerospace Reuters,Reuters Private investment firm Carlyle Group...
2,3,Oil and Economy Cloud Stocks Outlook Reuters,Reuters Soaring crude prices plus worriesabou...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters Authorities have halted oil exportflo...
4,3,Oil prices soar to alltime record posing new m...,AFP Tearaway world oil prices toppling record...
...,...,...,...
39995,2,Schumi Not my weekend,The seventimes Formula One champion spun out i...
39996,2,Australia leads in Davis Cup relegation tie,Australia s Todd Woodbridge s straight sets wi...
39997,1,Newspaper tipoff leads to terror arrests,Police have been given more time to question f...
39998,1,Sagan dies at,Francoise Sagan the rebellious French writer w...


## 2 задание. Стемминг и лемматизация

Стемминг (процесс приведения словоформ к их основе)

In [12]:
# Функция для стемминга
def stem_text(text):
    stemmer = PorterStemmer()
    tokens = word_tokenize(text)  # Токенизация текста
    stemmed_words = [stemmer.stem(word) for word in tokens]  # Стемминг каждого слова
    return ' '.join(stemmed_words)  # Объединение обратно в строку

# Применение функции очистки и стемминга к столбцам Title и Description
train_df['Title'] = train_df['Title'].apply(stem_text)
train_df['Description'] = train_df['Description'].apply(stem_text)

test_df['Title'] = test_df['Title'].apply(stem_text)
test_df['Description'] = test_df['Description'].apply(stem_text)


Лемматизация

In [13]:
# Инициализация лемматизатора
lemmatizer = WordNetLemmatizer()

# Функция для лемматизации текста
def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Применение лемматизации к столбцам Title и Description
train_df['Title'] = train_df['Title'].apply(lemmatize_text)
train_df['Description'] = train_df['Description'].apply(lemmatize_text)
test_df['Title'] = test_df['Title'].apply(lemmatize_text)
test_df['Description'] = test_df['Description'].apply(lemmatize_text)

Или так:

In [14]:
"""
import spacy

# Загрузка модели языка
nlp = spacy.load("en_core_web_sm")

# Функция для лемматизации текста
def lemmatize_text_spacy(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Применение лемматизации к столбцам Title и Description
train_df['Title'] = train_df['Title'].apply(lemmatize_text_spacy)
train_df['Description'] = train_df['Description'].apply(lemmatize_text_spacy)
test_df['Title'] = test_df['Title'].apply(lemmatize_text_spacy)
test_df['Description'] = test_df['Description'].apply(lemmatize_text_spacy)
"""

'\nimport spacy\n\n# Загрузка модели языка\nnlp = spacy.load("en_core_web_sm")\n\n# Функция для лемматизации текста\ndef lemmatize_text_spacy(text):\n    doc = nlp(text)\n    return \' \'.join([token.lemma_ for token in doc])\n\n# Применение лемматизации к столбцам Title и Description\ntrain_df[\'Title\'] = train_df[\'Title\'].apply(lemmatize_text_spacy)\ntrain_df[\'Description\'] = train_df[\'Description\'].apply(lemmatize_text_spacy)\ntest_df[\'Title\'] = test_df[\'Title\'].apply(lemmatize_text_spacy)\ntest_df[\'Description\'] = test_df[\'Description\'].apply(lemmatize_text_spacy)\n'

## 3 задание. Векторизация

Метод TF-IDF

In [15]:
# Объединение заголовков и описаний
train_combined = train_df['Title'] + " " + train_df['Description']
test_combined = test_df['Title'] + " " + test_df['Description']

# Векторизация с использованием TF-IDF
vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(train_combined)
tfidf_test = vectorizer.transform(test_combined)

# Преобразование в DataFrame для удобного отображения
tfidf_train_df = pd.DataFrame(tfidf_train.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_test_df = pd.DataFrame(tfidf_test.toarray(), columns=vectorizer.get_feature_names_out())

In [16]:
tfidf_train_df

,_not,aa,aaa,aai,aalborg,aan,aani,aapl,aaplnasdaq,aaplo,...,zurab,zurich,zurichnew,zvezda,zviadauri,zvidauri,zvonareva,zwick,zyman,zzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
tfidf_test_df

,_not,aa,aaa,aai,aalborg,aan,aani,aapl,aaplnasdaq,aaplo,...,zurab,zurich,zurichnew,zvezda,zviadauri,zvidauri,zvonareva,zwick,zyman,zzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
for i in range(4):
  # Удаление каждого второго столбца из tfidf_train_df
  tfidf_train_df = tfidf_train_df.iloc[:, ::2]

  # Удаление каждого второго столбца из tfidf_test_df
  tfidf_test_df = tfidf_test_df.iloc[:, ::2]

In [19]:
tfidf_train_df

,_not,ab,abbrevi,abdulahad,abigg,abort,abraham,absalon,abumoham,academia,...,zaguni,zambrano,zdnetcom,zeman,zesn,zib,zingarevich,zoeller,zoysa,zyman
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4 задание. Модель

In [20]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Преобразование данных в массивы numpy
X_train = tfidf_train_df.values
y_train = train_df['Class Index'].values
X_test = tfidf_test_df.values

# Преобразование меток классов в числовой формат (если необходимо)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_encoded, test_size=0.2, random_state=42)

# Определение параметров модели
input_dim = X_train.shape[1]  # количество признаков
output_dim = len(np.unique(y_train_encoded))  # количество классов

# Создание модели RNN
model = Sequential()
model.add(SimpleRNN(128, input_shape=(1, input_dim), return_sequences=False))  # 128 - размер скрытого слоя
model.add(Dense(output_dim, activation='softmax'))

# Компиляция модели
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Изменение формы входных данных для RNN (добавление размерности)
X_train_rnn = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val_rnn = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

# Обучение модели
model.fit(X_train_rnn, y_train, validation_data=(X_val_rnn, y_val), epochs=10, batch_size=32)

# Подготовка тестовых данных для предсказания
X_test_rnn = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Предсказание классов для тестового набора
predictions = model.predict(X_test_rnn)
predicted_classes = np.argmax(predictions, axis=1)

# Преобразование предсказанных классов обратно в оригинальные метки (если необходимо)
predicted_labels = label_encoder.inverse_transform(predicted_classes)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.4704 - loss: 1.2286 - val_accuracy: 0.5548 - val_loss: 1.0199
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5744 - loss: 0.9824 - val_accuracy: 0.5585 - val_loss: 1.0074
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5807 - loss: 0.9510 - val_accuracy: 0.5579 - val_loss: 1.0104
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.5796 - loss: 0.9421 - val_accuracy: 0.5535 - val_loss: 1.0127
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5852 - loss: 0.9294 - val_accuracy: 0.5508 - val_loss: 1.0165
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.5775 - loss: 0.9316 - val_accuracy: 0.5598 - val_loss: 1.0185
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.5856 - loss: 0.9218 - val_accuracy: 0.5489 - val_loss: 1.0282
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5821 - loss: 0.9211 

In [21]:
predicted_labels

array([3, 4, 4, ..., 2, 4, 4])

In [22]:
array_np = np.arange(7600)

In [23]:
sub = pd.core.frame.DataFrame({
    'ID': array_np,
    'Target': predicted_labels
})

In [24]:
sub

,ID,Target
0,0,3
1,1,4
2,2,4
3,3,2
4,4,4
...,...,...
7595,7595,2
7596,7596,2
7597,7597,2
7598,7598,4


In [25]:
sub.to_csv('lab2.csv', index=False)